In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf 

from kaggle_environments import evaluate, make
from kaggle_environments.envs.halite.helpers import *

from tf_agents.environments import py_environment
from tf_agents.environments import tf_py_environment
from tf_agents.trajectories import time_step as ts

from tf_agents.specs import BoundedArraySpec

from random import randint

Loading environment football failed: No module named 'gfootball'


# Hyperparameters

In [3]:
num_iterations = 60000 # @param {type:"integer"}

initial_collect_steps = 400  # @param {type:"integer"} 
collect_steps_per_iteration = 10  # @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 200  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}

epsilon_greedy = 0.1 # @param {type:"float"}

# Environment

In [4]:
def opponent_agent(obs, config, policy):
    board = Board(obs, config)
    me = board.opponents[0]
    
#     policy = tf.saved_model.load('my_policy')
    
    policy_obs = train_py_env.get_observation_v2(board, position=me.ships[0].position, player_id=me.id)
    
    if obs.step == 0:
        time_step = ts.restart(policy_obs)
    elif obs.step >= config.episodeSteps-1:
        time_step = ts.termination(policy_obs, reward=0)
    else:
        time_step = ts.transition(policy_obs, reward=0)
    
    time_step = time_step_ok(time_step)
    
    action_int = int(policy.action(time_step).action)
    action = train_py_env._action_def[action_int]
    
#     for ship in me.ships:
#         ship.next_action = action

    return action

In [7]:
class HaliteWrapper(py_environment.PyEnvironment):
    def __init__(self):
        
        # créer un environnement : où récupérer la config ? la renseigner directement ici ? 
        self._board_size = 7
        self._agent_count = 2
        self._max_turns = 20
        self._starting_halite = 2500
        self.env = make("halite", configuration={'episodeSteps':self._max_turns, 'size':self._board_size, 
                                                 'startingHalite':self._starting_halite})
        self.state = self.env.reset(num_agents=self._agent_count)
        
        self.obs = self.state[0].observation
        self.config = self.env.configuration
        self.max_cell_halite = self.config.maxCellHalite
        self.board = Board(self.obs, self.config)
                
        # observation_spec : va dépendre de la "vision" que l'on donne au vaisseau du plateau
        # action_spec 
#         self._channels = 2
        self.observation_shape = self.get_observation(self.board).shape
        self._observation_spec = BoundedArraySpec(
            shape=self.observation_shape, 
            dtype=np.int64, maximum=None, minimum=None)
        
        self._action_def = {0: None,
                            1: ShipAction.NORTH,
                            2: ShipAction.EAST,
                            3: ShipAction.SOUTH,
                            4: ShipAction.WEST,
#                             5: ShipAction.CONVERT
                           }
        self._action_spec = BoundedArraySpec(shape=(), dtype=np.int32, maximum=len(self._action_def)-1, minimum=0)
        
        # définir des variables de suivi : compteur de tours, halite au tour précédent ... En fonction des besoins
        # ATTENTION : Le turn counter indique ici le numéro du tour qui sera lancé au prochain step, pas le numéro du dernier tour résolu 
        self.turn_counter = 1
        self.previous_cargo_halite = 0
        self.game_over = False 
        
        self.opponent_policy = tf.saved_model.load('my_policy')
        
    def get_observation_v2(self, board, position=(0,0), player_id=0):
        size = board.configuration.size

        halite = np.array(board.observation['halite']).reshape(1, size, size)
        halite = np.tile(halite, (3,3))

        # convert Halite SDK position format to np matrix 
        row = size - 1 - position[1] + size
        col = position[0] + size
        radius = 3
        halite_radius = halite[0, row-radius+1:row+radius, col-radius+1:col+radius]

        # -1 for ally ships, +1 for opponent ships 
        ships = np.zeros((1, size, size))
        for player in board.players:
            if player != player_id:
                for ship in board.players[player].ships:
                    ships[0, size-1-ship.position[1], ship.position[0]] = 1
#             else:
#                 for ship in board.players[player].ships:
#                     ships[0, size-1-ship.position[1], ship.position[0]] = -1                
        
        ships = np.tile(ships, (3,3))
        ships_radius = ships[0, row-radius+1:row+radius, col-radius+1:col+radius]       
        
        # On pourra ajouter ships, ship_cargo et base 
        observation = np.concatenate([halite_radius, ships_radius], axis=0)
        observation = np.expand_dims(observation, axis=2) # added to be able to use Conv2D layers in q net preprocessing
        observation = np.array(observation, dtype='int64')
        return observation
    
    # Ajouter la position du vaisseau comme entrée ? 
    # Dans la fonction, ne pas utiliser self.board afin de pouvoir y faire appel en dehors de la classe
    #  plutôt à mettre dans un helper comme suggéré dans le post Kaggle
    def get_observation(self, board):
        size = board.configuration.size
        me = board.current_player
        
#         ships = np.zeros((1, size, size))
#         ship_cargo = np.zeros((1, size, size))
#         bases = np.zeros((1, size, size))

        # Possible de diviser par 1000 ou par max_halite afin d'avoir une valeur plus proche de 1
        halite = np.array(board.observation['halite']).reshape(1, size, size)
        halite = np.tile(halite, (3,3))
        
        if len(me.ships) == 0:
            ship_row = 0
            ship_col = 0
        else:
            ship = me.ships[0]
            ship_row = ship.position[1]
            ship_col = ship.position[0]
        
        # convert Halite SDK position format to np matrix 
        row = size - 1 - ship_row + size
        col = ship_col + size
        radius = 3
        halite_radius = halite[0, row-radius+1:row+radius, col-radius+1:col+radius]
        
        ships = np.zeros((1, size, size))
        for opponent in board.opponents:
            for ship in opponent.ships:
                ships[0, size-1-ship.position[1], ship.position[0]] = 1
        ships = np.tile(ships, (3,3))
        ships_radius = ships[0, row-radius+1:row+radius, col-radius+1:col+radius]
        
        # On pourra ajouter ships, ship_cargo et base 
        observation = np.concatenate([halite_radius, ships_radius], axis=0)
        observation = np.expand_dims(observation, axis=2) # added to be able to use Conv2D layers in q net preprocessing
        observation = np.array(observation, dtype='int64')
        return observation
    
    def observation_spec(self):
        return self._observation_spec
    
    def action_spec(self):
        return self._action_spec
    
    def _reset(self):
        # reset environment and board
        self.env = make("halite", configuration={'episodeSteps':self._max_turns, 'size':self._board_size, 
                                                 'startingHalite':self._starting_halite})
        self.state = self.env.reset(num_agents=self._agent_count)
        
        self.obs = self.state[0].observation
        self.config = self.env.configuration
        self.max_cell_halite = self.config.maxCellHalite
        self.board = Board(self.obs, self.config)
        
        # reset ALL counters
        self.turn_counter = 1
        self.previous_cargo_halite = 0
        self.game_over = False
        
        self.opponent_policy = tf.saved_model.load('my_policy')
        
        return_object = ts.restart(self.get_observation(self.board))
        
        return return_object
    
    def _step(self, action):
        
        if self.game_over:
            return self._reset()
        
        # Agent action
        action = int(action) # nécessaire ? normalement on a déjà fixé que c'était un int 
        self.board.ships['0-1'].next_action = self._action_def[action]
        
        # Opponent action
        self.board.ships['0-2'].next_action = opponent_agent(self.obs, self.config, self.opponent_policy)
#         self.board.ships['0-2'].next_action = self._action_def[randint(1,4)]
#         self.board.ships['0-2'].next_action = None
        
        self.board = self.board.next()
        
        # Calculate reward
        if len(self.board.players[0].ships) == 0: # ship destroyed
            reward = -10000
        elif len(self.board.players[1].ships) == 0: # enemy ship destroyed
            reward = 10000
        else:
#             reward = self.board.ships['0-1'].halite - self.previous_cargo_halite
            reward = self.board.ships['0-1'].halite - self.board.ships['0-2'].halite
        
        # Update all counters 
        self.turn_counter += 1
        if len(self.board.players[0].ships) > 0:
            self.previous_cargo_halite = self.board.ships['0-1'].halite
        else:
            self.previous_cargo_halite = 0
            
        if self.turn_counter >= self._max_turns:
            self.game_over = True
        elif len(self.board.players[0].ships) == 0 or len(self.board.players[1].ships) == 0:
            self.game_over = True
        
        # final
        if self.game_over:
            return_object = ts.termination(self.get_observation(self.board), reward)
            return return_object
        else:
            return_object = ts.transition(self.get_observation(self.board), reward, discount=1.0)
            return return_object

In [8]:
# Pour tester le fonctionnement 
# train_py_env = HaliteWrapper()
# print(train_py_env.board)
# train_py_env._step(action=0)

In [9]:
train_py_env = HaliteWrapper()
eval_py_env = HaliteWrapper()

In [10]:
train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

In [11]:
# Create a q_network based on the observation and action spec (ie, array shapes)
# For now, action will be a single command for a single ship 

from tf_agents.networks import q_network

preprocessing_layers = tf.keras.models.Sequential()
preprocessing_layers.add(tf.keras.layers.Conv2D(filters=8, kernel_size=(3,3), input_shape=(10,5,1), padding='same', activation='relu'))
preprocessing_layers.add(tf.keras.layers.Flatten())

fc_layer_params = (512,512)

# q_net = q_network.QNetwork(
#     input_tensor_spec=train_env.observation_spec(),
#     action_spec=train_env.action_spec(),
#     preprocessing_layers=preprocessing_layers,
#     fc_layer_params=fc_layer_params)

# Categorical version (for C51 DQN)
from tf_agents.networks import categorical_q_network

num_atoms = 51

q_net = categorical_q_network.CategoricalQNetwork(
    input_tensor_spec=train_env.observation_spec(),
    action_spec=train_env.action_spec(),
    preprocessing_layers=preprocessing_layers,
    num_atoms=num_atoms, 
    fc_layer_params=fc_layer_params)

In [12]:
from tf_agents.agents.dqn import dqn_agent
from tf_agents.utils import common

optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

# agent = dqn_agent.DqnAgent(
#     train_env.time_step_spec(),
#     train_env.action_spec(),
#     q_network=q_net,
#     optimizer=optimizer,
#     epsilon_greedy=epsilon_greedy,
#     td_errors_loss_fn=common.element_wise_squared_loss,
#     train_step_counter=train_step_counter)

# Categorical DQN (C51)
from tf_agents.agents.categorical_dqn import categorical_dqn_agent
agent = categorical_dqn_agent.CategoricalDqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    categorical_q_network=q_net,
    optimizer=optimizer,
    min_q_value=-1000, 
    max_q_value=+1000,
#     n_step_update=2,
    epsilon_greedy=epsilon_greedy,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

In [13]:
eval_policy = agent.policy
collect_policy = agent.collect_policy

In [14]:
from tf_agents.policies import random_tf_policy

random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())

In [15]:
def compute_avg_return(environment, policy, num_episodes=10):

    total_return = 0.0
    for _ in range(num_episodes):
        time_step = environment.reset()
        episode_return = 0.0

        while not time_step.is_last():
            action_step = policy.action(time_step)
            time_step = environment.step(action_step.action)
            episode_return += time_step.reward
        total_return += episode_return

    avg_return = total_return / num_episodes
    return avg_return.numpy()[0]

In [16]:
compute_avg_return(eval_env, random_policy, num_eval_episodes)

-2982.3

In [17]:
from tf_agents.replay_buffers import tf_uniform_replay_buffer

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_max_length)

from tf_agents.trajectories import trajectory

def collect_step(environment, policy, buffer):
    time_step = environment.current_time_step()
    action_step = policy.action(time_step)
    next_time_step = environment.step(action_step.action)
    traj = trajectory.from_transition(time_step, action_step, next_time_step)

    # Add trajectory to the replay buffer
    buffer.add_batch(traj)

def collect_data(env, policy, buffer, steps):
    for _ in range(steps):
        collect_step(env, policy, buffer)

collect_data(train_env, random_policy, replay_buffer, initial_collect_steps)

# Dataset generates trajectories with shape [Bx2x...]
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3, 
    sample_batch_size=batch_size, 
    num_steps=2).prefetch(3)

dataset

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


<PrefetchDataset shapes: (Trajectory(step_type=(64, 2), observation=(64, 2, 10, 5, 1), action=(64, 2), policy_info=(), next_step_type=(64, 2), reward=(64, 2), discount=(64, 2)), BufferInfo(ids=(64, 2), probabilities=(64,))), types: (Trajectory(step_type=tf.int32, observation=tf.int64, action=tf.int32, policy_info=(), next_step_type=tf.int32, reward=tf.float32, discount=tf.float32), BufferInfo(ids=tf.int64, probabilities=tf.float32))>

In [18]:
iterator = iter(dataset)

print(iterator)

In [ ]:
#@test {"skip": true}
try:
    %%time
except:
    pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

# Reset the train step
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
returns = [avg_return]

for _ in range(num_iterations):

    # Collect a few steps using collect_policy and save to the replay buffer.
    collect_data(train_env, agent.collect_policy, replay_buffer, collect_steps_per_iteration)

    # Sample a batch of data from the buffer and update the agent's network.
    experience, unused_info = next(iterator)
    train_loss = agent.train(experience).loss

    step = agent.train_step_counter.numpy()

    if step % log_interval == 0:
        print('step = {0}: loss = {1}'.format(step, train_loss))

    if step % eval_interval == 0:
        avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
        print('step = {0}: Average Return = {1}'.format(step, avg_return))
        returns.append(avg_return)

step = 200: loss = 0.5069378614425659
step = 400: loss = 0.666037380695343
step = 600: loss = 0.8509167432785034
step = 800: loss = 0.706379771232605
step = 1000: loss = 0.6270936727523804
step = 1000: Average Return = -5000.0
step = 1200: loss = 0.6803358197212219
step = 1400: loss = 0.8370119333267212
step = 1600: loss = 1.0100064277648926
step = 1800: loss = 0.9597554206848145
step = 2000: loss = 1.3083300590515137
step = 2000: Average Return = -991.7999877929688
step = 2200: loss = 0.8088275790214539
step = 2400: loss = 1.3424229621887207
step = 2600: loss = 1.56211519241333
step = 2800: loss = 1.5116565227508545
step = 3000: loss = 1.8050440549850464
step = 3000: Average Return = -980.0
step = 3200: loss = 1.754807710647583
step = 3400: loss = 2.6498630046844482
step = 3600: loss = 1.6210072040557861
step = 3800: loss = 2.3492560386657715
step = 4000: loss = 2.5898613929748535
step = 4000: Average Return = -1851.4000244140625
step = 4200: loss = 2.922142505645752
step = 4400: loss

In [ ]:
#@test {"skip": true}

import matplotlib.pyplot as plt

iterations = range(0, num_iterations + 1, eval_interval)
plt.plot(iterations, returns)
plt.ylabel('Average Return')
plt.xlabel('Iterations')
plt.ylim(top=4000)

plt.savefig('return_per_iteration.png', bbox_inches='tight')

In [35]:
# Pour visualiser le comportement de l'agent par print de boards successifs
# time_step = eval_env.reset()
# print(eval_py_env.board)

# while not time_step.is_last():
#     action_step = eval_policy.action(time_step)
#     print(eval_py_env._action_def[int(action_step.action)])
#     time_step = eval_env.step(action_step.action)
#     print(eval_py_env.board)

In [30]:
# Agent 

def time_step_ok(time_step_nok):
    tensors = []
    for array in time_step_nok:
        new_array = np.expand_dims(array, axis=0)
        tensor = tf.constant(new_array)
        tensors.append(tensor)
        
    # step_type
    # reward
    # discount
    # observation
    
    return ts.TimeStep(tensors[0], tensors[1], tensors[2], tensors[3])

def my_rl_agent(obs, config):
    board = Board(obs, config)
#     train_py_env.get_observation(board)
    me = board.current_player
    
    if obs.step == 0:
        time_step = ts.restart(train_py_env.get_observation(board))
    elif obs.step >= config.episodeSteps-1:
        time_step = ts.termination(train_py_env.get_observation(board), reward=0)
    else:
        time_step = ts.transition(train_py_env.get_observation(board), reward=0)
    
    time_step = time_step_ok(time_step)
    
    action_int = int(eval_policy.action(time_step).action)
    action = train_py_env._action_def[action_int]
    
    for ship in me.ships:
        ship.next_action = action

    return me.next_actions

In [31]:
def random_ship_agent(obs, config):
    board = Board(obs, config)
    me = board.current_player
    
    for ship in me.ships:
        ship.next_action = train_py_env._action_def[randint(0,4)]
    
    return me.next_actions

def static_ship_agent(obs, config):
    board = Board(obs, config)
    me = board.current_player
    
    return me.next_actions

def random_moving_ship_agent(obs, config):
    board = Board(obs, config)
    me = board.current_player
    
    for ship in me.ships:
        ship.next_action = train_py_env._action_def[randint(1,4)]
    
    return me.next_actions

In [51]:
train_py_env._reset()
env = train_py_env.env
env.run([my_rl_agent, my_rl_agent])
env.render(mode='ipython', width=400, height=300)

In [52]:
from tf_agents.policies.policy_saver import PolicySaver

In [53]:
saver = PolicySaver(agent.policy, batch_size=None)

In [54]:
saver.save('my_policy')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: my_policy\assets


INFO:tensorflow:Assets written to: my_policy\assets


In [17]:
load_policy = tf.saved_model.load('my_policy')

In [24]:
def my_loaded_rl_agent(obs, config):
    board = Board(obs, config)
    me = board.current_player
    
    policy_obs = train_py_env.get_observation_v2(board, position=me.ships[0].position, player_id=me.id)
    
    if obs.step == 0:
        time_step = ts.restart(policy_obs)
    elif obs.step >= config.episodeSteps-1:
        time_step = ts.termination(policy_obs, reward=0)
    else:
        time_step = ts.transition(policy_obs, reward=0)
    
    time_step = time_step_ok(time_step)
    
    action_int = int(load_policy.action(time_step).action)
    action = train_py_env._action_def[action_int]
    
    for ship in me.ships:
        ship.next_action = action

    return me.next_actions

In [29]:
train_py_env._reset()
env = train_py_env.env
env.run([my_loaded_rl_agent, my_loaded_rl_agent])
env.render(mode='ipython', width=400, height=300)

In [11]:
steps = train_py_env.env.run(['random', 'random'])

In [ ]:
# possible de faire un step sur l'environnement Kaggle 
env.step(actions={'0-1': 'NORTH'})

In [29]:
# Pour obtenir l'id du joueur actuel
obs.player

0

In [ ]:
# Possible de définir un agent qui prend en entrée un board plutot que obs et config
# On voit qu'on peut aussi fournir à l'environnement Kaggle l'agent sous forme de fonction plutôt que de fichier 

@board_agent
def move_ships_north_agent(board):
    for ship in board.current_player.ships:
        ship.next_action = ShipAction.NORTH

env.reset(2)
env.run([move_ships_north_agent, "random"])
env.render(mode="ipython", width=500, height=450)